<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/hazard_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 地域危険度一覧
- https://www.toshiseibi.metro.tokyo.lg.jp/bosai/chousa_6/home.htm


|項目|内容|
|:---|:---|
|建物倒壊危険度|建物倒壊の危険性|
|火災危険度|火災の発生による延焼の危険性|
|災害時活動困難係数|道路等の整備状況による災害時の活動の困難さ|
|総合危険度|建物倒壊危険量及び火災危険量を合算し、災害時活動困難係数を乗じて測定|

- 地盤分類
    - 地盤分類は、以下「地盤分類図」により、町丁目別に 12 種類の地盤に分類し、各地盤分類について、増幅率を設定して地盤の揺れやすさを示しています。
    - 増幅率の数値が大きいほど地盤が揺れやすいことを示します。

- 危険量（棟/ha）
    - 危険量は、地震による危険性の度合いを示すものです。
    - 「建物倒壊危険量（棟/ha）」は、地震による面積当たりの建物全壊棟数、「火災危険量（棟/ha）」は、地震による面積当たりの建物全焼棟数をそれぞれ示しています。
    - 「総合危険量（棟/ha）」は、「建物倒壊危険量（棟/ha）」、「火災危険量（棟/ha）」を合算し、地域レベルの道路整備状況などから評価した災害時活動困難係数を乗じたものです。
    - 危険量の数値が大きいほど、地震による危険性の度合いが高くなることを示します。
- 順位
    - 都内の市街化区域の 5,192 町丁目について、危険量の大きい町丁目から順位付けを行ったものです。
    - 順位の数値が小さくなるほど、地震による危険性の度合いが高くなることを示します。
- ランク
    - 地域危険度のランクは５段階の相対評価としています。
    - 各ランクの存在比率をあらかじめ定め、上記順位に基づき、ランクを割り当てています。
    - 1(危険度が低い) <---------> 5(危険度が高い)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [3]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse
import math
import io
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
     

In [4]:
def get_data_csv_url(url, encoding='utf-8', excel=False):
    res = requests.get(url)
    if res.status_code == 200:
        if not excel:
            return pd.read_csv(io.BytesIO(res.content), encoding=encoding)
        else:
            return pd.read_excel(io.BytesIO(res.content))
    else:
        raise Exception(f'status_code: {res.status_code} ({url})')

In [18]:
url_danger_zone = 'https://www.toshiseibi.metro.tokyo.lg.jp/bosai/chousa_6/download/all2.csv'
df_danger_zone = get_data_csv_url(url_danger_zone, encoding='cp932')
print(df_danger_zone.shape)

(5192, 13)


In [19]:
df_danger_zone.head()

,区市町名,町丁目名,地盤分類,建物倒壊危険度危険量（棟/ha）,建物倒壊危険度順位,建物倒壊危険度ランク,火災危険度危険量（棟/ha）,火災危険度順位,火災危険度ランク,災害時活動困難度困難係数,総合危険度危険量（棟/ha）,総合危険度順位,総合危険度ランク
0,千代田区,飯田橋１丁目,谷底低地3,2.01,1773,2,0.01,4067,1,0.11,0.22,2977,1
1,千代田区,飯田橋２丁目,谷底低地3,1.35,2598,2,0.00,4464,1,0.10,0.13,3743,1
2,千代田区,飯田橋３丁目,谷底低地3,0.52,4254,1,0.00,4646,1,0.10,0.05,4503,1
3,千代田区,飯田橋４丁目,谷底低地3,1.86,1927,2,0.01,4372,1,0.09,0.16,3458,1
4,千代田区,一番町,台地2,0.31,4573,1,0.00,4773,1,0.12,0.04,4654,1


In [20]:
town_name_trans = [
    ('千代田区', '神田三崎町', '三崎町'),
    ('千代田区', '神田猿楽町', '猿楽町'),
    ('中央区', '日本橋蛎殻町', '日本橋蠣殻町'),
    ('渋谷区', '鶯谷町', '鴬谷町'),
    ('渋谷区', '松濤', '松涛'),
    ('渋谷区', '富ヶ谷', '富ケ谷'),
    ('渋谷区', '千駄ヶ谷', '千駄ケ谷'),
    ('渋谷区', '幡ヶ谷', '幡ケ谷'),
    ('北区', '桐ケ丘', '桐ヶ丘'),
    ('北区', '西ケ原', '西ヶ原'),
    ('足立区', '入谷町', '入谷町，舎人町'),
]
df_danger_zone['町丁目'] = df_danger_zone['町丁目名']
for city, before, after in town_name_trans:
    df_danger_zone.loc[
        (df_danger_zone['区市町名'] == city) & (df_danger_zone['町丁目名'].str.contains(before)), '町丁目'
    ] = df_danger_zone['町丁目名'].str.replace(before, after)

In [21]:
df_danger_zone[
    df_danger_zone['町丁目名'] != df_danger_zone['町丁目']
]

,区市町名,町丁目名,地盤分類,建物倒壊危険度危険量（棟/ha）,建物倒壊危険度順位,建物倒壊危険度ランク,火災危険度危険量（棟/ha）,火災危険度順位,火災危険度ランク,災害時活動困難度困難係数,総合危険度危険量（棟/ha）,総合危険度順位,総合危険度ランク,町丁目
78,千代田区,神田猿楽町１丁目,台地1,0.56,4184,1,0.00,4548,1,0.09,0.05,4512,1,猿楽町１丁目
79,千代田区,神田猿楽町２丁目,台地1,1.20,2896,1,0.01,4158,1,0.11,0.14,3702,1,猿楽町２丁目
108,千代田区,神田三崎町１丁目,谷底低地3,0.46,4348,1,0.00,4605,1,0.13,0.06,4445,1,三崎町１丁目
109,千代田区,神田三崎町２丁目,谷底低地3,1.99,1795,2,0.02,3762,1,0.12,0.24,2814,2,三崎町２丁目
110,千代田区,神田三崎町３丁目,谷底低地3,1.06,3180,1,0.00,4585,1,0.11,0.12,3846,1,三崎町３丁目
159,中央区,日本橋蛎殻町１丁目,沖積低地3,4.41,720,3,0.08,2718,2,0.11,0.47,1595,2,日本橋蠣殻町１丁目
160,中央区,日本橋蛎殻町２丁目,沖積低地3,2.80,1269,2,0.03,3419,1,0.08,0.23,2926,1,日本橋蠣殻町２丁目
1630,渋谷区,鶯谷町,台地2,1.34,2617,2,0.03,3479,1,0.18,0.25,2767,2,鴬谷町
1652,渋谷区,松濤１丁目,台地2,0.63,4072,1,0.00,4519,1,0.11,0.07,4335,1,松涛１丁目
1653,渋谷区,松濤２丁目,台地2,0.70,3943,1,0.02,3937,1,0.10,0.07,4346,1,松涛２丁目


In [23]:
df_geocoding = pd.read_csv(os.path.join(DIR_NAME, 'geocoding.csv'))[['town_id', '市区町村名', '町丁目']]
print(df_geocoding.shape)

(3145, 3)


In [24]:
df_geocoding.head()

,town_id,市区町村名,町丁目
0,101003,千代田区,丸の内１丁目
1,101004,千代田区,丸の内２丁目
2,101005,千代田区,丸の内３丁目
3,101007,千代田区,大手町１丁目
4,101008,千代田区,大手町２丁目


In [31]:
df_danger_zone_merge = pd.merge(
    df_geocoding,
    df_danger_zone.rename(columns={'区市町名': '市区町村名'}).drop(columns=['町丁目名']),
    on=['市区町村名', '町丁目'],
    how='left'
)
print(df_danger_zone_merge.shape)
assert len(df_danger_zone_merge) == len(df_geocoding)

(3145, 14)


In [32]:
# マージできなかった町丁目
df_danger_zone_merge[df_danger_zone_merge['地盤分類'].isna()]

,town_id,市区町村名,町丁目,地盤分類,建物倒壊危険度危険量（棟/ha）,建物倒壊危険度順位,建物倒壊危険度ランク,火災危険度危険量（棟/ha）,火災危険度順位,火災危険度ランク,災害時活動困難度困難係数,総合危険度危険量（棟/ha）,総合危険度順位,総合危険度ランク
213,102123,中央区,水面,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,103147,港区,水面,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
919,108197,江東区,水面,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1050,109158,品川区,水面,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1354,111269,大田区,羽田沖水面,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1355,111271,大田区,多摩川河川敷（上流）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1356,111272,大田区,多摩川河川敷（下流）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1357,111273,大田区,ふるさとの浜辺公園,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
for col in ['建物倒壊危険度ランク', '火災危険度ランク', '総合危険度ランク']:
    df_danger_zone_merge[col] = df_danger_zone_merge[col].fillna(0).astype(int)

In [41]:
df_danger_zone_merge.to_csv(os.path.join(DIR_NAME, 'hazard_master.csv'), index=False, encoding='utf-8_sig')

In [42]:
df_danger_zone_merge['総合危険度ランク'].value_counts().sort_index()

0       8
1     953
2    1084
3     735
4     280
5      85
Name: 総合危険度ランク, dtype: int64

In [43]:
df_danger_zone_merge['建物倒壊危険度ランク'].value_counts().sort_index()

0       8
1     742
2    1202
3     820
4     288
5      85
Name: 建物倒壊危険度ランク, dtype: int64

In [44]:
df_danger_zone_merge['火災危険度ランク'].value_counts().sort_index()

0       8
1    1007
2    1067
3     694
4     284
5      85
Name: 火災危険度ランク, dtype: int64

In [48]:
df_danger_zone_merge[
    (df_danger_zone_merge['総合危険度ランク'] == 5) &
    (df_danger_zone_merge['建物倒壊危険度ランク'] == 5) & 
    (df_danger_zone_merge['火災危険度ランク'] == 5)
].sort_values('総合危険度順位').head(10)

,town_id,市区町村名,町丁目,地盤分類,建物倒壊危険度危険量（棟/ha）,建物倒壊危険度順位,建物倒壊危険度ランク,火災危険度危険量（棟/ha）,火災危険度順位,火災危険度ランク,災害時活動困難度困難係数,総合危険度危険量（棟/ha）,総合危険度順位,総合危険度ランク
2148,118017,荒川区,荒川６丁目,沖積低地3,12.00,20.0,5,19.01,2.0,5,0.30,9.36,1.0,5
2154,118024,荒川区,町屋４丁目,沖積低地4,14.54,8.0,5,14.96,6.0,5,0.31,9.06,2.0,5
2783,121323,足立区,柳原２丁目,沖積低地4,16.72,3.0,5,22.37,1.0,5,0.22,8.63,3.0,5
2657,121167,足立区,千住柳町,沖積低地4,14.95,5.0,5,13.14,9.0,5,0.30,8.53,4.0,5
744,107108,墨田区,京島２丁目,沖積低地4,19.52,1.0,5,17.77,3.0,5,0.21,7.95,5.0,5
737,107099,墨田区,墨田３丁目,沖積低地4,14.52,9.0,5,17.38,4.0,5,0.24,7.74,6.0,5
2642,121150,足立区,千住大川町,沖積低地4,12.43,19.0,5,12.19,10.0,5,0.29,7.22,7.0,5
887,108159,江東区,北砂４丁目,沖積低地5,11.67,26.0,5,11.59,13.0,5,0.31,7.11,8.0,5
742,107105,墨田区,押上３丁目,沖積低地4,11.58,30.0,5,14.66,7.0,5,0.27,7.10,9.0,5
2631,121137,足立区,関原２丁目,沖積低地5,11.16,37.0,5,13.48,8.0,5,0.28,6.78,10.0,5
